In [6]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

import matplotlib as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from skimage.io import imread
from tqdm import tqdm


In [7]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784',cache=False)

In [8]:
X = mnist.data.astype('float32')
Y = mnist.target.astype('int64')

In [9]:
X = X/255.0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [20]:
class CNNModule(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(1,32,(3,3),1)
    self.conv2 = nn.Conv2d(32,64,(3,3),1)
    self.ReLU = nn.ReLU()
    self.pool = nn.MaxPool2d(stride=(2,2),kernel_size=(2,2))
    self.linear1 = nn.Linear(64*28*28,120)
    self.linear2 = nn.Linear(120,10)

  def forward(self,x):
    return self.linear2(self.ReLU(self.linear1(self.ReLU(self.conv2(self.conv1(x))))))




In [13]:
def accuracy_fn(y_true,y_pred):
  correct = torch.eq(y_true,y_pred).sum().item()
  acc = (correct/len(y_pred))*100
  return acc

In [21]:
model_0 = CNNModule()

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params = model_0.parameters(),lr = 0.1)

In [23]:
torch.manual_seed(42)


epochs=1000



for epoch in range(epochs):
  model_0.train()

  y_logits = model_0(X_train)
  y_pred = torch.round(torch.sigmoid(y_logits))

  loss = loss_fn(y_logits,
                 y_train)
  
  acc = accuracy_fn(y_true = y_train,
                    y_pred = y_pred)
  
  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  model_0.eval()
  with torch.inference_mode():
    test_logits = model_0(X_test).squeeze()
    test_preds =torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_logits,y_test)
    test_acc = accuracy_fn(y_true = y_test,
                           y_pred = test_preds)
    
  if epoch %10 ==0:
    print(f"Epoch:{epoch} |Loss:{loss:.5f}| Acc:{acc:.2f}% | Test loss:{test_loss:.5f} | Test acc:{test_acc:.2f}%")
     

TypeError: ignored